# core

> Fill in a module description here

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from datetime import datetime
import inspect

from web3 import Web3, AsyncWeb3

In [ ]:
#| export
class MultiRPCWeb3:
    """Web3 object that tries to execute a method in multiple RPCs until one succeeds."""

    def __init__(self, *rpcs, providers=None):
        self.rpcs = rpcs
        self.providers = providers
        if not self.providers:
            self.providers = []
            for rpc in self.rpcs:
                if hasattr(rpc, 'provider'):
                    self.providers.append(rpc.provider)

    @classmethod 
    def from_rpcs(obj, *rpcs):
        return obj(*[Web3(Web3.HTTPProvider(rpc)) for rpc in rpcs])

    @classmethod 
    def async_from_rpcs(obj, *rpcs):
        return obj(*[AsyncWeb3(AsyncWeb3.AsyncHTTPProvider(rpc)) for rpc in rpcs])
        

    def __getattr__(self, attr):

        def wrapper(*args, **kwargs):
            for i, (provider, rpc) in enumerate(zip(self.providers, self.rpcs)):
                try:
                    return rpc.__getattribute__(attr)(*args, **kwargs)
                except Exception as e:
                    print(f'RPC {provider} failed')
                    if i < len(self.providers) - 1:
                        print(f'Trying RPC {self.providers[i+1]}')
                    else:
                        print(f"All the RPCs failed to execute '{attr}'")
                        raise e

        async def awrapper(*args, **kwargs):
            for i, (provider, rpc) in enumerate(zip(self.providers, self.rpcs)):
                try:
                    return await rpc.__getattribute__(attr)(*args, **kwargs)
                except Exception as e:
                    print(f'RPC {provider} failed')
                    if i < len(self.providers) - 1:
                        print(f'Trying RPC {self.providers[i+1]}')
                    else:
                        print(f"All the RPCs failed to execute '{attr}'")
                        raise e

        for rpc in self.rpcs:
            rpc_attr = rpc.__getattribute__(attr)
            if callable(rpc_attr) and not inspect.iscoroutinefunction(rpc_attr):
                return wrapper
            elif inspect.iscoroutinefunction(rpc_attr):
                return awrapper
            elif attr in rpc.__dict__:
                return MultiRPCWeb3(*[rpc.__getattribute__(attr) for rpc in self.rpcs], providers=self.providers)
            elif attr in self.__dict__:
                return self.__getattribute__(attr)
        raise AttributeError(f"Attribute '{attr}' not found in any of the RPCs")

In [ ]:
w3 = MultiRPCWeb3.from_rpcs('http://rpc1:1235', 'http://rpc2:2345')

In [ ]:
block_number = w3.eth.get_block('latest')['number']
block_number

In [ ]:
#| export
class BlockTimestamps:
    """List like object timestamp[block_number] with cache"""
    def __init__(self, w3):
        self.w3 = w3
        self.cache = {}

    def __getitem__(self, block_number):
        if block_number in self.cache:
            return self.cache[block_number]
        block = self.w3.eth.get_block(block_number)
        if block_number=='latest':
            block_number = block['number']
        self.cache[block_number] = block['timestamp']
        return block['timestamp']

In [ ]:
block_timestamps = BlockTimestamps(w3)

In [ ]:
#| export 
def interpolation_search(o, timestamp, low=None, high=None, how='left'):
    """Interpolation search algorithm to find the block number of a timestamp in a BlockTimestamps object."""

    assert how in ['left', 'right']

    if low is None:
        # first PoS block
        low = 15537394
    if high is None:
        high_timestamp = o['latest']
        high = max(o.cache.keys())
    
    assert o[low] <= timestamp <= o[high]

    pos = low + ((int(timestamp) - o[low]) * (high - low)) // (o[high] - o[low])

    if o[pos] < timestamp:
        if o[pos + 1] > timestamp:
            if how=='left':
                return pos
            else:
                return pos + 1
        return interpolation_search(o, timestamp, pos + 1, high, how=how)
    else:
        if o[pos - 1] < timestamp:
            if how=='right':
                return pos
            else:
                return pos - 1
        return interpolation_search(o, timestamp, low, pos - 1, how=how)

In [ ]:
interpolation_search(block_timestamps, 1692548195)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()